In [44]:
library(tidyverse)

s6061 <- read_csv("./csv/1960-61.csv", show_col_types = FALSE)
s6162 <- read_csv("./csv/1961-62.csv", show_col_types = FALSE)
s6263 <- read_csv("./csv/1962-63.csv", show_col_types = FALSE)
s6364 <- read_csv("./csv/1963-64.csv", show_col_types = FALSE)
s6465 <- read_csv("./csv/1964-65.csv", show_col_types = FALSE)
s6566 <- read_csv("./csv/1965-66.csv", show_col_types = FALSE)
s6667 <- read_csv("./csv/1966-67.csv", show_col_types = FALSE)
s6768 <- read_csv("./csv/1967-68.csv", show_col_types = FALSE)
s6869 <- read_csv("./csv/1968-69.csv", show_col_types = FALSE)
s6970 <- read_csv("./csv/1969-70.csv", show_col_types = FALSE)
s7071 <- read_csv("./csv/1970-71.csv", show_col_types = FALSE)
s7172 <- read_csv("./csv/1971-72.csv", show_col_types = FALSE)
s7273 <- read_csv("./csv/1972-73.csv", show_col_types = FALSE)
s7374 <- read_csv("./csv/1973-74.csv", show_col_types = FALSE)
s7475 <- read_csv("./csv/1974-75.csv", show_col_types = FALSE)
s7576 <- read_csv("./csv/1975-76.csv", show_col_types = FALSE)
s7677 <- read_csv("./csv/1976-77.csv", show_col_types = FALSE)
s7778 <- read_csv("./csv/1977-78.csv", show_col_types = FALSE)
s7879 <- read_csv("./csv/1978-79.csv", show_col_types = FALSE)
s7980 <- read_csv("./csv/1979-80.csv", show_col_types = FALSE)
s8081 <- read_csv("./csv/1980-81.csv", show_col_types = FALSE)
s8182 <- read_csv("./csv/1981-82.csv", show_col_types = FALSE)
s8283 <- read_csv("./csv/1982-83.csv", show_col_types = FALSE)
s8384 <- read_csv("./csv/1983-84.csv", show_col_types = FALSE)
s8485 <- read_csv("./csv/1984-85.csv", show_col_types = FALSE)
s8586 <- read_csv("./csv/1985-86.csv", show_col_types = FALSE)
s8687 <- read_csv("./csv/1986-87.csv", show_col_types = FALSE)
s8788 <- read_csv("./csv/1987-88.csv", show_col_types = FALSE)
s8889 <- read_csv("./csv/1988-89.csv", show_col_types = FALSE)
s8990 <- read_csv("./csv/1989-90.csv", show_col_types = FALSE)
s9091 <- read_csv("./csv/1990-91.csv", show_col_types = FALSE)
s9192 <- read_csv("./csv/1991-92.csv", show_col_types = FALSE)
s9293 <- read_csv("./csv/1992-93.csv", show_col_types = FALSE)
s9394 <- read_csv("./csv/1993-94.csv", show_col_types = FALSE)
s9495 <- read_csv("./csv/1994-95.csv", show_col_types = FALSE)
s9596 <- read_csv("./csv/1995-96.csv", show_col_types = FALSE)

results <- bind_rows(s6061, s6162, s6263, s6364, s6465, s6566, s6667, s6768, s6869, s6970, s7071, s7172, s7273, s7374, s7475, s7576, s7677, s7778, s7879, s7980, s8081, s8182, s8283, s8384, s8586, s8687, s8788,s8889, s8990, s9091, s9192, s9293, s9394, s9495, s9596) %>%
    group_by(
        season
    ) %>%
    mutate(
        game_no = row_number()
    ) %>%
    select(season, game_no, scorers)

In [45]:
players <- read_csv("../output/player_ssns.csv", show_col_types = FALSE) %>%
    select(
        season,
        surname,
        player_name,
        disam_name
    ) %>%
    unique()

In [46]:
# Pivot scorers to wide format
scorers <- separate_wider_delim(
    results,
    scorers,
    delim = ",",
    names_sep = "",
    too_few = "align_start"
) %>%
mutate(
    across(starts_with("scorers"), str_squish)
  ) %>%
  pivot_longer(
    starts_with("scorers"),
    values_to = "scorer"
  ) %>%
  filter(
    !is.na(scorer)
  ) %>%
mutate(
  scorer_name = case_when(
    str_detect(scorer, "\\s\\d+$") ~ str_replace_all(scorer, "(?:\\s\\d+)?$", ""),
    TRUE ~ scorer
  ),
  goals_scored = case_when(
    str_detect(scorer, "\\s\\d+$") ~ str_extract(scorer, "\\d+$") %>% as.numeric(),
    TRUE ~ 1
  )
 ) %>%
left_join(
  players,
  by = c("season" = "season", "scorer_name" = "disam_name")
) %>%
group_by(
  season,
  game_no
) %>%
mutate(
  goals_for = sum(goals_scored),
  player_name = case_when(
    scorer_name == "OG" ~ "OG",
    .default = player_name
  )
)

# Check for players with missing full names
scorers %>% ungroup() %>% filter(is.na(player_name)) %>%
select(season, scorer_name) %>%
  unique() %>% arrange(scorer_name)

season,scorer_name
<chr>,<chr>


In [47]:
# Import results data
results_df <- read_csv("https://raw.githubusercontent.com/petebrown/update-results/main/data/results_df.csv", show_col_types = FALSE) %>%
  rename(game_no = ssn_game_no) %>%
  select(season, game_no, game_date, opposition, venue, score, goals_for, goals_against, generic_comp, game_type, league_tier, manager)

# Check for games where no of goals scores differs from scorer data
results_df %>%
  inner_join(
    scorers,
    by = c("season", "game_no")
  ) %>%
  select(
    season, game_no, game_date, opposition, venue, score, goals_for.x, goals_for.y
  ) %>%
  filter(
    goals_for.x != goals_for.y
  ) %>%
  unique()

season,game_no,game_date,opposition,venue,score,goals_for.x,goals_for.y
<chr>,<dbl>,<date>,<chr>,<chr>,<chr>,<dbl>,<dbl>


In [48]:
# Import Soccerbase goal data
goals_2 <- read_csv("https://raw.githubusercontent.com/petebrown/update-player-stats/main/data/players_df.csv", show_col_type = FALSE) %>%
    select(game_date, player_name, pl_goals) %>%
    filter(pl_goals > 0) %>%
    left_join(results_df %>% select(game_date, season, game_no), by = "game_date") %>%
    rename(
        goals_scored = pl_goals
    ) %>%
    select(season, game_no, player_name, goals_scored) %>%
    arrange(season, game_no)

In [49]:
# Bind new dataframes with Soccerbase data
scorers_long <- scorers %>%
    select(
        season, game_no, player_name, goals_scored
    ) %>%
    bind_rows(goals_2)

write_csv(scorers_long, "../output/scorers-long.csv")

In [50]:
# Join scorers to results
all_scorers <- scorers_long %>%
  left_join(results_df, by = c("season", "game_no"))

head(all_scorers, 3)

season,game_no,player_name,goals_scored,game_date,opposition,venue,score,goals_for,goals_against,generic_comp,game_type,league_tier,manager
<chr>,<dbl>,<chr>,<dbl>,<date>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<chr>
1960/61,1,Graham Barnett,1,1960-08-20,Brentford,A,1-4,1,4,Football League,League,3,Peter Farrell
1960/61,2,Ken Finney,1,1960-08-22,Swindon Town,H,2-2,2,2,Football League,League,3,Peter Farrell
1960/61,2,Graham Barnett,1,1960-08-22,Swindon Town,H,2-2,2,2,Football League,League,3,Peter Farrell


In [51]:
# Pivot to wide format for attachment to results
scorers_wide <- scorers %>%
    select(season, game_no, player_name, goals_scored) %>%
    mutate(
        scorer = paste(player_name, goals_scored, sep = " ")
    ) %>%
    group_by(season, game_no) %>%
    summarise(
        scorers = paste(scorer, collapse = ", "),
        goals_for = sum(goals_scored),
        .groups = "drop"
    ) %>%
    mutate(
        scorers = str_replace_all(scorers, " 1", "")
    ) %>%
    select(season, game_no, scorers)

head(scorers_wide, 3)

write_csv(scorers_wide, "../output/scorers_wide.csv")

season,game_no,scorers
<chr>,<int>,<chr>
1960/61,1,Graham Barnett
1960/61,2,"Ken Finney, Graham Barnett"
1960/61,5,"Ken Finney, Elkanah Onyeali 2, Tommy Eglington"


In [52]:
all_scorers %>%
    filter(
        game_type == "League"
    ) %>%
    group_by(
        season,
        player_name
    ) %>%
    summarise(
        goals_scored = sum(goals_scored),
        .groups = "drop"
    ) %>%
    ungroup() %>%
    group_by(
        season
    ) %>%
    mutate(
        pc_gls = round((goals_scored / sum(goals_scored)) * 100, 1)
    ) %>%
    arrange(
        desc(pc_gls),
        season,
        player_name
    ) %>%
    group_by(season) %>%
    slice_max(
        goals_scored,
        n = 3
    ) %>%
    ungroup() %>%
    arrange(
        desc(pc_gls), desc(goals_scored), season, player_name
    ) %>%
    head(10)

season,player_name,goals_scored,pc_gls
<chr>,<chr>,<dbl>,<dbl>
2018/19,James Norwood,29,46.8
1987/88,Ian Muir,27,44.3
1995/96,John Aldridge,27,42.2
1991/92,John Aldridge,22,39.3
1975/76,Ronnie Moore,34,38.2
2013/14,Ryan Lowe,19,37.3
1967/68,George Yardley,23,37.1
1986/87,Ian Muir,20,37.0
1994/95,John Aldridge,24,35.8


In [53]:
all_scorers %>%
filter(
    game_type == "League",
    league_tier < 5    
) %>%
group_by(season, player_name) %>%
summarise(
    goals = sum(goals_scored),
    .groups = "drop"
) %>%
arrange(
    desc(goals),
    season,
    player_name
) %>%
filter(
    goals >= 20
) %>%
filter(
    !player_name %in% c("John Aldridge", "Ian Muir")
)

season,player_name,goals
<chr>,<chr>,<dbl>
1975/76,Ronnie Moore,34
1965/66,Barry Dyson,30
1964/65,Barry Dyson,29
2018/19,James Norwood,29
1960/61,Keith Williams,27
1963/64,Barry Dyson,26
1961/62,Alan Arnell,23
1964/65,Charlie McDonnell,23
1967/68,George Yardley,23


In [56]:
mans_10 <- results_df %>%
    group_by(manager) %>%
    summarise(
        games = n()
    ) %>%
    filter(games >= 10)

all_scorers %>%
    filter(
        game_type == "League",
        manager %in% mans_10$manager
    ) %>%
    group_by(
        manager,
        player_name
    ) %>%
    summarise(
        goals_scored = sum(goals_scored),
        .groups = "drop"
    ) %>%
    ungroup() %>%
    group_by(
        manager
    ) %>%
    mutate(
        pc_gls = round((goals_scored / sum(goals_scored)) * 100, 1)
    ) %>%
    arrange(
        desc(pc_gls),
        manager,
        player_name
    ) %>%
    filter(
        manager == "John King"
    )

manager,player_name,goals_scored,pc_gls
<chr>,<chr>,<dbl>,<dbl>
John King,John Aldridge,112,12.3
John King,Ian Muir,108,11.9
John King,Ronnie Moore,70,7.7
John King,Chris Malkin,59,6.5
John King,Johnny Morrissey,36,4.0
John King,Steve Peplow,36,4.0
John King,Russell Allen,32,3.5
John King,Jim Steel,29,3.2
John King,Pat Nevin,28,3.1
